In [44]:
import os
import pickle
import time
import numpy
import math
from aips import get_engine
from aips.data_loaders.outdoors import load_dataframe
from pyspark import SparkConf
from sentence_transformers import SentenceTransformer, SimilarityFunction
from pyspark.sql import SparkSession

engine = get_engine()
#Recommended for making ALS run faster, if you have enough memory / cores allocated to docker
conf = SparkConf()
conf.set("spark.driver.memory", "8g")
conf.set("spark.executor.memory", "8g")
conf.set("spark.dynamicAllocation.enabled", "true")
conf.set("spark.dynamicAllocation.executorMemoryOverhead", "8g")
spark = SparkSession.builder.appName("AIPS-ch13").config(conf=conf).getOrCreate()

model = SentenceTransformer("mixedbread-ai/mxbai-embed-large-v1",
                            similarity_fn_name=SimilarityFunction.DOT_PRODUCT,
                            truncate_dim=1024)

#https://github.com/facebookresearch/faiss/wiki/Pre--and-post-processing
#https://github.com/facebookresearch/faiss/wiki
#https://huggingface.co/spaces/sentence-transformers/quantized-retrieval/blob/main/app.py

In [45]:
![ ! -d 'outdoors' ] && git clone --depth=1 https://github.com/ai-powered-search/outdoors.git
! cd outdoors && git pull
! cd outdoors && cat outdoors.tgz.part* > outdoors.tgz
! cd outdoors && mkdir -p '../data/outdoors/' && tar -xvf outdoors.tgz -C '../data/outdoors/'

#outdoors_collection = engine.create_collection("outdoors")

Already up to date.
README.md
concepts.pickle
._guesses.csv
guesses.csv
._guesses_all.json
guesses_all.json
outdoors_concepts.pickle
outdoors_embeddings.pickle
._outdoors_golden_answers.csv
outdoors_golden_answers.csv
._outdoors_golden_answers.xlsx
outdoors_golden_answers.xlsx
._outdoors_golden_answers_20210130.csv
outdoors_golden_answers_20210130.csv
outdoors_labels.pickle
outdoors_question_answering_contexts.json
outdoors_questionanswering_test_set.json
outdoors_questionanswering_train_set.json
._posts.csv
posts.csv
predicates.pickle
pull_aips_dependency.py
._question-answer-seed-contexts.csv
question-answer-seed-contexts.csv
question-answer-squad2-guesses.csv
._roberta-base-squad2-outdoors
roberta-base-squad2-outdoors/
roberta-base-squad2-outdoors/._tokenizer_config.json
roberta-base-squad2-outdoors/tokenizer_config.json
roberta-base-squad2-outdoors/._special_tokens_map.json
roberta-base-squad2-outdoors/special_tokens_map.json
roberta-base-squad2-outdoors/._config.json
roberta-base-

### Note: I updated the get_embeddings method earlier in the chapter to have the same method signature, so no need to duplicate it in the manuscript. Ideally we'd load this in from a python file for reuse.

In [46]:
#model.stop_multi_process_pool(pool)
#pool = model.start_multi_process_pool()
#embeddings = model.encode(texts, convert_to_tensor=False).tolist()

def get_embeddings(texts, model, cache_name, ignore_cache=False):
    cache_file_name = f"data/embeddings/{cache_name}.pickle"
    if ignore_cache or not os.path.isfile(cache_file_name):        
        embeddings = model.encode(texts, normalize_embeddings=True)
        os.makedirs(os.path.dirname(cache_file_name), exist_ok=True)
        with open(cache_file_name, "wb") as fd:
            pickle.dump(embeddings, fd)
    else:
        with open(cache_file_name, "rb") as fd:
            embeddings = pickle.load(fd)
    return embeddings

## Boilerplate code for Quantization listings
### Generating embeddings and benchmark data

In [47]:
import faiss
from aips.data_loaders.outdoors import load_dataframe

def display_results(scores, ids, data):
    results = generate_search_results(scores, ids, data)
    display(results)
    return results

def get_outdoors_data():
    outdoors_dataframe = load_dataframe("data/outdoors/posts.csv")
    outdoors_data = list(outdoors_dataframe.rdd.map(lambda r: r.asDict()).collect())
    return outdoors_data

def display_statistics(full_search_results, quantized_search_results, start_message="Recall"):
    index_name = quantized_search_results["index_name"]
    full_search_time = full_search_results["time_taken"]
    time_taken = quantized_search_results["time_taken"]
    time_imp = round((full_search_time - time_taken) * 100 / full_search_time, 2)
    quantized_size = quantized_search_results["size"]
    improvement_ms = f"({time_imp}% improvement)"
    improvement_size = f"({round((full_search_results['size'] - quantized_size) * 100 / full_search_results['size'], 2)}% improvement)"
    print(f"{index_name} search took: {time_taken:.3f} ms {improvement_ms}")
    print(f"{index_name} index size: {round(quantized_size / 1000000, 2)} MB {improvement_size}")
    recall = calculate_recall(full_search_results["results"], quantized_search_results["results"])
    print(f"{start_message}: {str(round(recall, 4))}")

def calculate_recall(scored_full_results, scored_quantized_results):
    recalls = []
    for i in range(len(scored_full_results)):
        full_ids = [r["id"] for r in scored_full_results[i]]
        quantized_ids = [r["id"] for r in scored_quantized_results[i]]
        recalls.append((len(set(full_ids).intersection(set(quantized_ids))) /
                       len(set(quantized_ids))))
    return sum(recalls) / len(recalls)

def generate_search_results(faiss_scores, faiss_ids):
    outdoors_data = get_outdoors_data()
    faiss_results = []
    for i in range(len(faiss_scores)):
        results = []
        for j, id in enumerate(faiss_ids[i]):
            id = int(id)
            result = {"score": faiss_scores[i][j],
                      "title": outdoors_data[id]["title"],
                      "body": outdoors_data[id]["body"],
                      "id": id}
            results.append(result)
        faiss_results.append(results)
    return faiss_results

## Listing 13.20

### Indexing full-precision embeddings using FAISS

In [48]:
from sentence_transformers.quantization import quantize_embeddings

model = SentenceTransformer("mixedbread-ai/mxbai-embed-large-v1",
                            similarity_fn_name=SimilarityFunction.DOT_PRODUCT,
                            truncate_dim=1024)

def index_full_precision_embeddings(doc_embeddings, name):
    index = faiss.IndexFlatIP(doc_embeddings.shape[1])
    index.add(doc_embeddings)
    faiss.write_index(index, name)
    return index

def get_outdoors_embeddings(model):
    outdoors_dataframe = load_dataframe("data/outdoors/posts.csv")
    post_texts = [post["title"] + " " + post["body"]
                  for post in outdoors_dataframe.collect()]
    return numpy.array(get_embeddings(post_texts, model, "outdoors_mrl_normed")) #TODO: This will take 2-5 hours to run the first time if not cached. Upload to github to save readers hassle.


doc_embeddings = get_outdoors_embeddings(model) #takes 2-3 hours if not cached
full_index = index_full_precision_embeddings(doc_embeddings, "full_embeddings")

## Listing 13.21

### Generating full-precision query embeddings

In [49]:
def format_query_for_encoding(query):
    #return "Represent this sentence for searching relevant passages: " + query
    return query
    
def get_test_queries():
    queries = ["tent poles", "hiking trails", "mountain forests",
            "white water", "best waterfalls", "mountain biking",
            "snowboarding slopes", "bungee jumping", "public parks"]
    return list(map(format_query_for_encoding, queries))

queries = get_test_queries()
print(queries)
query_embeddings = model.encode(queries, convert_to_numpy=True,
                                normalize_embeddings=True)

['tent poles', 'hiking trails', 'mountain forests', 'white water', 'best waterfalls', 'mountain biking', 'snowboarding slopes', 'bungee jumping', 'public parks']


## Listing 13.22

### Functions to benchmark quantized search approaches

In [50]:
def time_and_execute_search(index, index_name, query_embeddings, k=25):
    start_time = time.time()
    faiss_scores, faiss_ids = index.search(query_embeddings, k=k)
    time_taken = ((time.time() - start_time) * 1000)
    
    results = {"results": generate_search_results(faiss_scores, faiss_ids),
               "time_taken": time_taken, 
               "faiss_scores": faiss_scores, "faiss_ids": faiss_ids}
    index_stats = {}
    if index_name:
        index_stats ={"index_name": index_name,
                      "size": os.path.getsize(index_name)}
    return results | index_stats


def evaluate_search(full_index, quantized_index, quantized_index_name, query_embeddings, quantized_query_embeddings,
                    k=25, display=True, log=False):
    full_results = time_and_execute_search(full_index, "full_embeddings", query_embeddings, k=k)
    quantized_results = time_and_execute_search(quantized_index, quantized_index_name,
                                                quantized_query_embeddings, k=k)
    if display:
        display_statistics(full_results, quantized_results)
    return quantized_results, full_results

def evaluate_rerank_search(full_index, quantized_index, query_embeddings,
                           quantized_embeddings, k=50, limit=25):
    results, full_results = evaluate_search(full_index, quantized_index, None, query_embeddings,
                                            quantized_embeddings, display=False, k=k)
    
    doc_embeddings = get_outdoors_embeddings(model) #This can point to a cheap on-disk data source containing the original full-precision embeddings
    rescore_scores, rescore_ids = [], []
    for i in range(len(results["results"])):
        embedding_ids = results["faiss_ids"][i]
        top_k_embeddings = [doc_embeddings[id] for id in embedding_ids]
        query_embedding = query_embeddings[i]
        scores = query_embedding @ numpy.array(top_k_embeddings).T
        indices = scores.argsort()[::-1][:limit]
        top_k_indices = embedding_ids[indices]
        top_k_scores = scores[indices]
        rescore_scores.append(top_k_scores)
        rescore_ids.append(top_k_indices)

    results = generate_search_results(rescore_scores, rescore_ids)
    recall = calculate_recall(full_results["results"], results)
    print(f"Reranked recall: {recall}")

## Listing 13.23
### int8 quantization

In [59]:
def index_int8_embeddings(doc_embeddings, name):
    int8_embeddings = quantize_embeddings(doc_embeddings, precision="int8")
    print("Int8 embeddings shape:", int8_embeddings.shape)
    index = faiss.IndexScalarQuantizer(int8_embeddings.shape[1],
                                       faiss.ScalarQuantizer.QT_8bit, )
    index.train(int8_embeddings)
    index.add(int8_embeddings)
    faiss.write_index(index, name)
    return index

int8_index_name = "int8_embeddings"
int8_index = index_int8_embeddings(doc_embeddings, int8_index_name)

quantized_queries = quantize_embeddings(query_embeddings,
                                        calibration_embeddings=doc_embeddings,
                                        precision="int8")

evaluate_search(full_index, int8_index, int8_index_name, query_embeddings, quantized_queries)
evaluate_rerank_search(full_index, int8_index, query_embeddings, quantized_queries)

Int8 embeddings shape: (18456, 1024)
int8_embeddings search took: 69.244 ms (-949.73% improvement)
int8_embeddings index size: 18.91 MB (74.99% improvement)
Recall: 0.9289
Reranked recall: 1.0


## Listing 13.24
### Binary Quantization

In [61]:
def index_binary_embeddings(doc_embeddings, binary_index_name):
    binary_embeddings = quantize_embeddings(doc_embeddings,
                                    precision="binary").astype(numpy.uint8)
    print("Binary embeddings shape:", binary_embeddings.shape)
    index = faiss.IndexBinaryFlat(binary_embeddings.shape[1] * 8)
    index.add(binary_embeddings)
    faiss.write_index_binary(index, binary_index_name)
    return index

binary_index_name = "binary_embeddings"
binary_index = index_binary_embeddings(doc_embeddings, binary_index_name)

quantized_queries = quantize_embeddings(query_embeddings,
                        calibration_embeddings=doc_embeddings,
                                       precision="binary").astype(numpy.uint8)

evaluate_search(full_index, binary_index, binary_index_name, query_embeddings, quantized_queries)
evaluate_rerank_search(full_index, binary_index, query_embeddings, quantized_queries)

Binary embeddings shape: (18456, 128)
binary_embeddings search took: 5.629 ms (96.27% improvement)
binary_embeddings index size: 2.36 MB (96.87% improvement)
Recall: 0.6044
Reranked recall: 1.0


## Listing 13.25
### Product Quantization

In [66]:
def index_pq_embeddings(doc_embeddings, index_name):
    dimensions = doc_embeddings.shape[1]
    M_subvectors = 64
    num_bits = 8
    index = faiss.IndexPQ(dimensions, M_subvectors, num_bits)
    index.train(doc_embeddings)
    index.add(doc_embeddings)   
    faiss.write_index(index, index_name) # Commit the index to disk
    return index

pq_index_name = "pq_embeddings"
pq_index = index_pq_embeddings(doc_embeddings, pq_index_name)

evaluate_search(full_index, pq_index, pq_index_name, query_embeddings, query_embeddings)
evaluate_rerank_search(full_index, pq_index, query_embeddings, query_embeddings)

#TODO: consider adding IVFFlatPQ as optimization for speed to show at end

pq_embeddings search took: 12.890 ms (92.49% improvement)
pq_embeddings index size: 2.23 MB (97.05% improvement)
Recall: 0.5778
Reranked recall: 0.991111111111111


## Listing 13.26
### Matryoshka Representations Learning

In [88]:
def get_mrl_embeddings(embeddings, num_dimensions):
    mrl_embeddings = numpy.array(list(map(lambda e: e[:num_dimensions], embeddings)))
    return mrl_embeddings

def index_mrl_embeddings(doc_embeddings, num_dimensions, mrl_index_name):
    mrl_doc_embeddings = get_mrl_embeddings(doc_embeddings, num_dimensions)
    print(f"{mrl_index_name} embeddings shape:", mrl_doc_embeddings.shape)
    mrl_index = index_full_precision_embeddings(mrl_doc_embeddings, mrl_index_name)
    return mrl_index

print(f"Original embeddings shape:", doc_embeddings.shape)
original_dimensions = doc_embeddings.shape[1] #1024

for num_dimensions in [original_dimensions//2, #512
                       original_dimensions//4, #256
                       original_dimensions//8]: #128
    print("\n")
    mrl_index_name = f"mrl_embeddings_{num_dimensions}"
    mrl_index = index_mrl_embeddings(doc_embeddings, num_dimensions, mrl_index_name)
    mrl_queries = get_mrl_embeddings(query_embeddings, num_dimensions)

    evaluate_search(full_index, mrl_index, mrl_index_name, query_embeddings, mrl_queries)
    evaluate_rerank_search(full_index, mrl_index, query_embeddings, mrl_queries)

Original embeddings shape: (18456, 1024)


mrl_embeddings_512 embeddings shape: (18456, 512)
mrl_embeddings_512 search took: 4.079 ms (77.53% improvement)
mrl_embeddings_512 index size: 37.8 MB (50.0% improvement)
Recall: 0.7022
Reranked recall: 1.0


mrl_embeddings_256 embeddings shape: (18456, 256)
mrl_embeddings_256 search took: 2.212 ms (74.8% improvement)
mrl_embeddings_256 index size: 18.9 MB (75.0% improvement)
Recall: 0.4756
Reranked recall: 0.9688888888888888


mrl_embeddings_128 embeddings shape: (18456, 128)
mrl_embeddings_128 search took: 1.592 ms (81.34% improvement)
mrl_embeddings_128 index size: 9.45 MB (87.5% improvement)
Recall: 0.2489
Reranked recall: 0.64


## Listing 13.27

### Combining techniques

In [97]:
def get_mrl_embeddings(embeddings, num_dimensions):
    mrl_embeddings = numpy.array(list(map(lambda e: e[:num_dimensions], embeddings)))
    return mrl_embeddings

def index_binary_ivf_halffmrl_embeddings(doc_embeddings, num_dimensions, binary_index_name):

    #MRL
    half_mrl_doc_embeddings =  get_mrl_embeddings(doc_embeddings, num_dimensions)
    
    #Binary quantization
    binary_embeddings = quantize_embeddings(half_mrl_doc_embeddings,
                                    calibration_embeddings=half_mrl_doc_embeddings,
                                    precision="binary").astype(numpy.uint8)
    d = half_mrl_doc_embeddings.shape[1]
    quantizer = faiss.IndexBinaryFlat(d)
    
    #ANN: IVF Flat Algorithm
    index = faiss.IndexBinaryIVF(quantizer, d, nlist)
    index.nprobe = 4

    index.train(binary_embeddings)
    index.add(binary_embeddings)
    faiss.write_index_binary(index, binary_index_name)
    return index

mrl_dimensions = doc_embeddings.shape[1] // 2  #MRL 1024 => 512 dimensions
binary_ivf_halffmrl_index_name = "binary_ivf_halfmrl_embeddings"
binary_ivf_halffmrl_index = index_binary_ivf_halffmrl_embeddings(doc_embeddings, mrl_dimensions, binary_ivf_halffmrl_index_name)

half_mrl_doc_embeddings =  get_mrl_embeddings(doc_embeddings, mrl_dimensions)
halfmrl_queries = get_mrl_embeddings(query_embeddings, mrl_dimensions)
quantized_queries = quantize_embeddings(halfmrl_queries,
                        calibration_embeddings=half_mrl_doc_embeddings,
                                       precision="binary").astype(numpy.uint8)

evaluate_search(full_index, binary_ivf_halffmrl_index, binary_ivf_halffmrl_index_name, query_embeddings, quantized_queries)
evaluate_rerank_search(full_index, binary_ivf_halffmrl_index, query_embeddings, quantized_queries)

binary_ivf_halfmrl_embeddings search took: 0.807 ms (91.78% improvement)
binary_ivf_halfmrl_embeddings index size: 1.35 MB (98.22% improvement)
Recall: 0.3511
Reranked recall: 0.7244444444444444
